In [1]:
from injector_surrogate_quads import Surrogate_NN
from pytorch_injector_surrogate_model import *
import torch
import torch.nn as nn

import sys,os
sys.path.append('../configs')
#Sim reference point to optimize around
from ref_config import ref_point

/Users/conniexu/opt/anaconda3/envs/baxenv/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Pytorch Injector Surrogate Model Example

### This notebook shows how to create and use Pytorch versions of the original Keras/Tf injector surrogate model. 

In [2]:
# load keras/tf injector model
Model = Surrogate_NN()

Model.load_saved_model(model_path = '../models/', \
                       model_name = 'model_OTR2_NA_rms_emit_elu_2021-07-27T19_54_57-07_00')
Model.load_scaling()
Model.take_log_out = False

# print(Model.model_1.summary())

### direct creation of pytorch model ###
test_model = Surrogate_NN_PyTorch(Model)
test_model.eval()

model_name = 'Surrogate_NN_PyTorch'
model_path = '../models/' + model_name + '.pth'
if os.path.exists(model_path) == False and os.stat(model_path).st_size <= 0: 
    print(f"saving {model_name}")
    torch.save(test_model, '../models/Surrogate_NN_PyTorch.pth')

2022-07-28 10:41:28.825933: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
### load existing pytorch model with Surrogate_NN ### 
PyTorch_Model = Surrogate_NN(pytorch=True)

PyTorch_Model.load_saved_model(model_path = '../models/', 
                       model_name = 'Surrogate_NN_PyTorch')

PyTorch_Model.load_scaling(scalerfilex = '../data/transformer_x_pytorch.pth', 
                   scalerfiley = '../data/transformer_y_pytorch.pth')

PyTorch_Model.take_log_out = False

print(PyTorch_Model)

In [4]:
import numpy as np

transformer_x_name = 'transformer_x_pytorch'
transformer_y_name = 'transformer_y_pytorch'

transformer_x_path = '../data/' + transformer_x_name + '.pth'
transformer_y_path = '../data/' + transformer_y_name + '.pth'

# save pytorch versions of injector surrogate model's transformers 
if os.path.exists(transformer_x_path) == False and os.stat(transformer_x_path).st_size <= 0: 
    print(f'saving {transformer_x_name} and {transformer_y_name} to data')
    transformer_x = MinMaxScaler_PyTorch(Model.transformer_x)
    transformer_y = MinMaxScaler_PyTorch(Model.transformer_y)

    torch.save(transformer_x, transformer_x_path)
    torch.save(transformer_y, transformer_y_path)

# verify that original injector surrogate model and new pytorch model are the same
keras_test_point = Model.sim_to_machine(np.asarray(ref_point))
torch_test_point = torch.as_tensor(Model.sim_to_machine(ref_point))
# print(keras_test_point)
# print(torch_test_point)

# original surrogate 
keras_out = Model.pred_machine_units(keras_test_point)
print(f'orig: {keras_out}')

# PyTorch surrogate
torch_out = PyTorch_Model.pred_machine_units(torch_test_point)
print(f'pytorch: {torch_out.detach()}')


2022-07-28 10:41:28.992849: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)


orig: [[1.2124054e-04 1.1827494e-04 4.6100092e-04 5.5473919e-07 5.6714555e-07]]
pytorch: tensor([[1.2124e-04, 1.1827e-04, 4.6100e-04, 5.5474e-07, 5.6715e-07]])
